1. How do you debug a pod stuck in CrashLoopBackoff?

In [ ]:
--> “If a pod is stuck in CrashLoopBackOff, I usually follow a structured debugging process. First, I check the pod’s status with kubectl get pods to confirm the issue.
--> Then I run kubectl describe pod <pod-name> to see events, like whether it’s failing due to liveness/readiness probes, OOMKilled, or some other scheduling issue.
--> Next, I check the container logs using kubectl logs <pod-name>, and if the pod is restarting too quickly, I use the --previous flag to capture logs from the previous crash. [kubectl logs <pod-name> --previous
] if pod as multiple container [kubectl logs <pod-name> -c <container-name> --previous]
Based on experience, common causes are application errors, misconfigured environment variables, wrong commands/entrypoints in the image, failed probes, or resource constraints. For example, if the logs show ‘OOMKilled’, I would increase memory limits. If the liveness probe is failing, I would adjust the probe configuration or fix the app’s startup.
If logs don’t help, I use kubectl debug or override the entrypoint with a sleep command so I can exec into the container and inspect configs or dependencies.
So my approach is: check describe, check logs, validate probes and resource limits, and then fix the underlying application or configuration issue before redeploying.”*

In [ ]:
2.How will you stop pod in k8s?

In [ ]:
--> *“In Kubernetes, we normally don’t stop a pod directly, because pods are managed by higher-level controllers like Deployments, ReplicaSets, or StatefulSets. If I just delete the pod using kubectl delete pod <pod-name>, the controller will immediately recreate it to maintain the desired replicas.
If I really want to stop the pod permanently, 

--> I need to scale down the controller (for example, kubectl scale deployment <name> --replicas=0). That way, Kubernetes won’t recreate the pod.
cmd: kubectl scale deployment sindhu-deployment  --replicas=0 -n default 

--> If it’s a standalone pod without a controller, deleting it with kubectl delete pod will stop it.

--> For debugging, I can also temporarily ‘stop’ a pod by running kubectl drain on the node, or by setting replicas=0 in the YAML manifest.”*

In [ ]:
3. how will you replicate a pod ?

In [ ]:
--> *“In Kubernetes, we don’t usually replicate a single pod manually because pods are ephemeral and managed by controllers. Instead, we use controllers like Deployments, ReplicaSets, or StatefulSets to handle replication automatically.
    For example, if I have a pod defined in a Deployment, I can increase the number of replicas by updating the replicas field:*

--> kubectl scale deployment <deployment-name> --replicas=3

--> *This ensures Kubernetes maintains 3 copies of the pod. If a pod crashes, the controller automatically recreates it.

--> For standalone pods (not managed by a controller), I would need to manually create additional pods with the same specification, but that’s not recommended in production.”*

In [ ]:
4. command to get logs in k8s?

In [ ]:
“To check logs in Kubernetes, I use kubectl logs <pod-name>.
 If the pod has multiple containers, I specify the container with -c <container-name>. 
 For pods that are crashing repeatedly, I use the --previous flag to see logs from the last crashed instance.
 And if the pod is in a different namespace, I add -n <namespace>.”

In [ ]:
5.what will you do if pod is getting more load and we need to stay it healthy before it gets died?

In [ ]:
“If a pod is experiencing high load and we want to keep it healthy, I would handle it in multiple ways:

Horizontal Scaling: Increase the number of pod replicas using a Deployment or ReplicaSet. For example:*

kubectl scale deployment <deployment-name> --replicas=3

Or set up a Horizontal Pod Autoscaler (HPA) to automatically scale based on CPU/memory load:

kubectl autoscale deployment <deployment-name> --cpu-percent=50 --min=2 --max=10

Vertical Scaling: Increase the resources (CPU/memory) allocated to the pod in its YAML spec to handle higher load.

Load Distribution: Ensure the service is properly load-balanced across the pods.

Readiness/Liveness Probes: Check that probes are configured correctly so Kubernetes can restart unhealthy pods before they fail completely.

So my approach is proactive: scale horizontally, ensure sufficient resources, and configure proper probes to maintain pod health under high load.”

In [ ]:
6. How do you manage multiple environments (dev, stage, prod)?

Separate namespaces or clusters.

Use Helm or Kustomize for environment-specific configs.

In [ ]:
"Auto-generated Terraform code is useful to bootstrap,
 but it’s not production-ready. It usually contains hardcoded values,
  lacks variables and modules, and doesn’t follow best practices like state management or secrets handling.
   We should refactor it into reusable modules with variables, 
  outputs, and remote backends before using it for real infrastructure."